# Please don't forget to upvote if you find it somehow helpful! (This version is not for submission because of timeout. Still working on this problem, welcome any possible solution.)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#!pip install datatable > /dev/null
import datatable as dt
import matplotlib.pyplot as plt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploratory Data Analysis

1. Since the data is related with time, a straight forward guess would be that maybe the variable that we are predicting is distributed related with time. However, I tested this guess by running autocorrelation,and plotting the trend of total 'actions' per day. Seems there is no obvious relation between time and 'action'. We can assume that 'action' is only related with the features.
2. There are lots of excellent work in visualzing the trend of features. We do not have extra information about features except for a table relating features and tags. One possible way of grouping the features with tags could be truncated SVD, which can decrease the number of tag groups. You can check out this method at this  website.[https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html](http://) I tried this method but it works not as good as visualizing them and grouping. Therefore, I used Carl McBride Ellis's grouping method. https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance We only need to focus one response variable in resp, resp1, resp2, resp3, resp4. On the other hand, the features are seperately related with resp, resp1,etc. I focus on features related with resp.

In [ ]:
##starting timer
import time
start = time.time()
start

In [ ]:

train = dt.fread('../input/jane-street-market-prediction/train.csv').to_pandas()

##transform resp to action
train['action']=(train['resp']>0).astype('int')


##grouping features(this step is from carlmcbrideellis's work)
grouplists=[['feature_7','feature_9','feature_11','feature_13','feature_15'],['feature_8','feature_10','feature_12','feature_14','feature_16'],
            ['feature_17','feature_19','feature_21','feature_23','feature_25'],['feature_18','feature_20','feature_22','feature_24','feature_26'],
            ['feature_27','feature_29','feature_31','feature_33','feature_35'],['feature_28','feature_30','feature_32','feature_34','feature_36'],
            ['feature_55','feature_56','feature_57','feature_58','feature_59'],['feature_72','feature_73','feature_74','feature_75','feature_76'],
            ['feature_78','feature_79','feature_80','feature_81','feature_82'],['feature_84','feature_85','feature_86','feature_87','feature_88'],
            ['feature_90','feature_91','feature_92','feature_93','feature_94'],['feature_96','feature_97','feature_98','feature_99','feature_100'],
            ['feature_102','feature_103','feature_104','feature_105','feature_106'],['feature_108','feature_109','feature_110','feature_111','feature_112'],
            ['feature_114','feature_115','feature_116','feature_117','feature_118'],['feature_120','feature_122','feature_124','feature_126','feature_128'],
            ['feature_121','feature_123','feature_125','feature_127','feature_129']]
resp_group=[]
features=['feature_7','feature_9','feature_11','feature_13','feature_15','feature_8','feature_10','feature_12','feature_14','feature_16',
            'feature_17','feature_19','feature_21','feature_23','feature_25','feature_18','feature_20','feature_22','feature_24','feature_26',
            'feature_27','feature_29','feature_31','feature_33','feature_35','feature_28','feature_30','feature_32','feature_34','feature_36',
            'feature_55','feature_56','feature_57','feature_58','feature_59','feature_72','feature_73','feature_74','feature_75','feature_76',
            'feature_78','feature_79','feature_80','feature_81','feature_82','feature_84','feature_85','feature_86','feature_87','feature_88',
            'feature_90','feature_91','feature_92','feature_93','feature_94','feature_96','feature_97','feature_98','feature_99','feature_100',
            'feature_102','feature_103','feature_104','feature_105','feature_106','feature_108','feature_109','feature_110','feature_111','feature_112',
            'feature_114','feature_115','feature_116','feature_117','feature_118','feature_120','feature_122','feature_124','feature_126','feature_128',
            'feature_121','feature_123','feature_125','feature_127','feature_129']
for i,groups in enumerate(grouplists):
    resp_group.append(groups[4])

In [ ]:
tr_plt=train['action'].groupby(train['date']).sum()
fig, ax = plt.subplots(figsize=(8,3))
ax.set_xlabel ("time", fontsize=18)
ax.set_ylabel ("action", fontsize=18)
ax.set_title("Action Trend", fontsize=20)
tr_plt.plot(lw=3)

import matplotlib.pyplot as plt
import statsmodels.api as sm
sm.graphics.tsa.plot_acf(tr_plt, lags=24)
plt.title('Autocorrelation of action', fontsize=18)
plt.xlabel('Lag', fontsize=18)
plt.ylabel('Autocorrelation', fontsize=18)
plt.show()

**Following is the pre-processing of data. Scaling features could make a difference when fitting a XGBoost model.**

In [ ]:
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

##feature scaling
from sklearn import preprocessing
scaler=preprocessing.StandardScaler().fit(train[resp_group])
train1=scaler.transform(train[resp_group])
train1=pd.DataFrame(train1)


t=pd.concat([train1,train['date'],train['weight'],train['action']],ignore_index=True,axis=1)

##focus on data points of which weight is not 0
train2=t[t[18]!=0]
train2.columns=['feature_15',
 'feature_16',
 'feature_25',
 'feature_26',
 'feature_35',
 'feature_36',
 'feature_59',
 'feature_76',
 'feature_82',
 'feature_88',
 'feature_94',
 'feature_100',
 'feature_106',
 'feature_112',
 'feature_118',
 'feature_128',
 'feature_129','date','weight','action']
train2.head()

try several ways to accelerate Tuning XGBoost 
import time
import xgboost as xgb
1.parallel jobs
evaluate the effect of the number of threads of XGBoosting process
results = []
num_threads = [1, 2, 3, 4,5,6]
for n in num_threads:
    start = time.time()
    model = xgb.XGBClassifier()(nthread=n)
    model.fit(train[resp_group][0:200], train['action'][0:200])
    elapsed = time.time() - start
    print(n, elapsed)
    results.append(elapsed)
plot results

plt.plot(num_threads, results)
plt.ylabel('Speed (seconds)')
plt.xlabel('Number of Threads')
plt.title('XGBoost Training Speed vs Number of Threads')
plt.show()

In [ ]:
train2['action']

**Tune hyper parameter for xgboost using grid search method.**

**tune hyper parameter for xgboost**

import warnings
warnings.filterwarnings('ignore')
train3=train2[train2['date']<20]
import xgboost as xgb
xgb1 = xgb.XGBClassifier(nthread=3)
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(train3[resp_group], train3['action'], test_size=0.1, random_state=13)

parameters = {'objective':['binary:logistic'],
              'learning_rate': [.001,.01,0.03],
              'max_depth': [8,12,16],
              'tree_method':['gpu_hist'],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs=3,
                        verbose=True,scoring='roc_auc')
xgb_grid.fit(x_train,y_train)

result=pd.DataFrame.from_dict(xgb_grid.cv_results_).sort_values(by=['rank_test_score'])
result.head(1)

In order to avoid seperating dataset when data are from the same day, I used this special spliting method to split into train set and test set. https://www.kaggle.com/aimind/bottleneck-encoder-mlp-keras-tuner-8601c5#PurgedGroupTimeSeriesSplit

https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

#source:
# https://www.kaggle.com/aimind/bottleneck-encoder-mlp-keras-tuner-8601c5#PurgedGroupTimeSeriesSplit
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
#train3=train2[train2['date']<55]
train3=train[train['date']<55]
param = { 'max_depth':16,
          'learning_rate':0.001}
import warnings
warnings.filterwarnings('ignore')
X=train3[features].values
y=train3['action'].values
#x_train, x_test, y_train, y_test = train_test_split(train2, train2['action'], test_size=0.1, random_state=13)
gkf = PurgedGroupTimeSeriesSplit(n_splits = 2, group_gap=20)
splits = list(gkf.split(y, groups=train3['date'].values))

xgb_models=[]
for fold, (train_indices, test_indices) in enumerate(splits):
    
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    trainset=xgb.DMatrix(X_train,y_train)
    clf=xgb.train(param, trainset)
    xgb_models.append(clf)
    X_test=xgb.DMatrix(X_test)
    predictions = clf.predict(X_test)
    predi=np.where(predictions >= 0.5, 1, 0).astype(int)
    print(classification_report(y_test,predi))

**Encode with autoencoder**

In [ ]:
##encode with autoencoder 
##source from https://www.tensorflow.org/tutorials/generative/autoencoder,
##https://www.tensorflow.org/api_docs/python/tf/keras/layers

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, GaussianNoise
from tensorflow import keras
def Autoencoder(latent_dim,input_dim,output_dim):
    inputs=Input(input_dim)
    encode=BatchNormalization()(inputs)
    encode = GaussianNoise(0.05)(encode)
    for i in range(1,2):
        encode=Dense(latent_dim, activation='relu')(encode)
        encode=Dropout(0.2)(encode)
    decoded=Dense(output_dim, activation='sigmoid')(encode)

    autoencoder = Model(inputs=inputs,outputs=decoded)

    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder

In [ ]:
##limit days in dataset when using neural network method
train3=train[train['date']<60]
train3=train3.dropna()
x_train, x_test, y_train, y_test = train_test_split(train3[features], train3['action'], test_size=0.2, random_state=13)
x_train, x_test, y_train, y_test=x_train.values, x_test.values, y_train.values, y_test.values
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))
latent_dim=256
input_dim=x_train.shape[-1]
output_dim=y_train.shape[-1]
autoencoder=Autoencoder(latent_dim,input_dim,output_dim)
autoencoder.fit(x_train,y_train,epochs=100,batch_size=4800,shuffle=True,validation_data=(x_test,y_test))
autoencoder.save_weights('./autoencoder_weights')

**MLP method(include encode layer with trained encoder in the above step)**

In [ ]:
x_train.shape[-1]
y_train.shape[-1]

In [ ]:
from keras.layers import Dense, Activation, Dropout, advanced_activations,Concatenate
from keras import backend as K
import keras

train3=train3.dropna()
X=train3[features].values
y=train3['action'].values
gkf = PurgedGroupTimeSeriesSplit(n_splits = 3, group_gap=20)
splits = list(gkf.split(y, groups=train3['date'].values))
def mlp(input_dim,output_dim,autoencoder):
    inputs=Input(input_dim)
    encoded=autoencoder(inputs)
    encoded = Concatenate()([encoded,inputs])
    normalized=BatchNormalization()(encoded)
    x=Dense(128)(normalized)
    x=Activation('tanh')(x)
    x=Dropout(0.2)(x)
    #for i in range(1,2):
    x = tf.keras.layers.Dense(128)(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x=Dense(1, activation='sigmoid')(x)

    mlp_model=Model(inputs=inputs,outputs=x)

    keras.initializers.he_normal(seed=None)
    mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy')
                  #metrics=
    return mlp_model

#for i in range(0,3):
#    model=mlp(input_dim,output_dim,autoencoder)
#    model.load_weights(f'../input/weights/mlp_{i}_timecv.index')
#    models.append(model)
models=[]
FOLDS = 3
SEED = 30
for fold, (train_indices, test_indices) in enumerate(splits):
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    input_dim=X_train.shape[-1]
    output_dim=y_train.shape[-1]
    #model.save_weights(f'./model_{SEED}_{fold}.hdf5')
    #model.compile(Adam(hp.get('lr')/100),loss='mse')
    model=mlp(input_dim,output_dim,autoencoder)
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=48,batch_size=4800)
    #model.fit(X_test,y_test,epochs=3,batch_size=4096)
    models.append(model)
    model.save_weights(f'./mlp_{fold}_timecv')
    

In [ ]:
for fold, (train_indices, test_indices) in enumerate(splits):    
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    predicted=models[0](X_test, training=False)
    predi=np.where(predicted >= 0.5, 1, 0).astype(int)
    print(classification_report(y_test,predi))

In [ ]:
##end time
end=time.time()
end-start ##time in seconds

# Thank you!